# Introduction

The purpose of this notebook is to do some EDA. The data was grabbed from https://www.kaggle.com/c/m5-forecasting-accuracy/data.

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.tabular import *

In [3]:
data_path = Path('./data/')
data_path.ls()

[PosixPath('data/merge_2.csv'),
 PosixPath('data/sell_prices.csv'),
 PosixPath('data/sales_df'),
 PosixPath('data/merge.csv'),
 PosixPath('data/calendar.csv'),
 PosixPath('data/calendar_temp.csv'),
 PosixPath('data/sales_train_validation.csv'),
 PosixPath('data/sales_temp.csv'),
 PosixPath('data/sample_submission.csv'),
 PosixPath('data/calendar_df')]

## Data

The competition includes four csv files:

* calendar.csv - Contains information about the dates on which the products are sold.

* sales_train_validation.csv - Contains historical daily unit sales data per product and store (d_1 - d_1913)

* sell_prices.csv - Contains information about the proces of products sold per store and data

* sample_submission.csv - Correct format for submission.

### Calendar.csv

The data columns are defined below:

**date**: The date in a “y-m-d” format. 

**wm_yr_wk**: The id of the week the date belongs to. 

**weekday**: The type of the day (Saturday, Sunday, …, Friday). 

**wday**: The id of the weekday, starting from Saturday. 

**month**: The month of the date. 

**year**: The year of the date. 

**event_name_1**: If the date includes an event, the name of this
    event. 

**event_type_1**: If the date includes an event, the type of 
    this event. 

**event_name_2**: If the date includes a second event, the name of 
    this event. 

**event_type_2**: If the date includes a second event, the type 
    of this event. 

**snap_CA, snap_TX, and snap_WI**: A binary variable (0 or 1) 
    indicating whether the stores of CA, TX or WI allow 
    SNAP[3] purchases on the examined date. 1 indicates that 
    SNAP purchases are allowed. 

In [4]:
calendar_df = pd.read_csv(data_path/'calendar.csv')
calendar_df

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [18]:
calendar_df[calendar_df.event_name_1.isnull() == False]['event_type_1'].unique()

array(['Sporting', 'Cultural', 'National', 'Religious'], dtype=object)

**Takeaways**: Categorize the relevant columns, add date part, and add elpased info.

### sales_train_validation.csv

The data columns are described below:

**item_id**: The id of the product. 

**dept_id**: The id of the department the product belongs to. 

**cat_id**: The id of the category the product belongs to. 

**store_id**: The id of the store where the product is sold. 

**state_id**: The State where the store is located. 

**d_1, d_2, …, d_i, … d_1941**: The number of units sold at day i, starting from 2011-01-29.  

In [5]:
sales_df = pd.read_csv(data_path/'sales_train_validation.csv')
sales_df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


**Takeaways**: Looks like the the data for units sold on a particular day is in
columns. Will have to create a new table with this broken into rows. Plus will have to categorize a few columns. Look into pandas' melt.

### sell_prices.csv

The data columns are described below:

**store_id**: The id of the store where the product is sold.  

**item_id**: The id of the product. 

**wm_yr_wk**: The id of the week. 

**sell_price**: The price of the product for the given week/store. The price is provided per week (average across seven days). If not available, this means that the product was not sold during the examined week. Note that although prices are constant at weekly basis, they may change through time (both training and test set).   

In [6]:
prices_df = pd.read_csv(data_path/'sell_prices.csv')
prices_df

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.00
6841117,WI_3,FOODS_3_827,11618,1.00
6841118,WI_3,FOODS_3_827,11619,1.00
6841119,WI_3,FOODS_3_827,11620,1.00


**Takeaways**: Looks like this data can just be attached to the sales df.

### submission.csv

**Takeaways**: The id column in a concat of item_id and store_id. Will have to generate forecast for f1-f28 and place in a csv formatted like this.

In [29]:
sub_df = pd.read_csv(data_path/'sample_submission.csv')
sub_df

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Data Prep/Feature Engineering

The three csv files can be converted into one. This new csv will have the following columns:

* id 	
* item_id
* dept_id
* cat_id
* store_id
* state_id
* date
* plus all of the date part
* elapsed cols
* wm_yr_wk
* units sold

In [7]:
#use df melt to create rows for all d_{i} columns
sales_df = sales_df.melt(id_vars=['id', 'item_id', 'dept_id', 
                                'cat_id', 'store_id', 'state_id'])

In [8]:
#rename col 'variable' to 'd'
sales_df = sales_df.rename(columns={'variable': 'd'})

In [9]:
sales_df

,id,item_id,dept_id,cat_id,store_id,state_id,d,value
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...,...
58327365,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1913,1
58327366,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1913,0
58327367,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1913,0
58327368,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1913,3


In [ ]:
#do this later.

#change d_{i} to actual dates and rename col to date
sales_df['d'] = sales_df['d'].map(lambda x: calendar_df[calendar_df['d'] == x]['date'].values[0])

#rename col 'variable' to 'd'
sales_df = sales_df.rename(columns={'d': 'date'})

### Feature Engineering - Calendar 

In [88]:
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [85]:
#add datepart cols
add_datepart(calendar_df, 'date', drop=False)

In [86]:
calendar_df

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,...,29,5,29,False,False,False,False,False,False,1296259200
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,...,30,6,30,False,False,False,False,False,False,1296345600
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,...,31,0,31,True,False,False,False,False,False,1296432000
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,...,1,1,32,False,True,False,False,False,False,1296518400
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,...,2,2,33,False,False,False,False,False,False,1296604800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,...,15,2,167,False,False,False,False,False,False,1465948800
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,...,16,3,168,False,False,False,False,False,False,1466035200
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,...,17,4,169,False,False,False,False,False,False,1466121600
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,...,18,5,170,False,False,False,False,False,False,1466208000


In [105]:
calendar_df['event_name_1'] = calendar_df.event_name_1.fillna('none').astype(np.str)
calendar_df['event_name_2'] = calendar_df.event_name_2.fillna('none').astype(np.str)
calendar_df['event_type_1'] = calendar_df.event_type_1.fillna('none').astype(np.str)
calendar_df['event_type_2'] = calendar_df.event_type_2.fillna('none').astype(np.str)

In [116]:
calendar_df.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,2011-01-29,11101,Saturday,1,1,2011,d_1,none,none,none,...,29,5,29,False,False,False,False,False,False,1296259200
1,2011-01-30,11101,Sunday,2,1,2011,d_2,none,none,none,...,30,6,30,False,False,False,False,False,False,1296345600
2,2011-01-31,11101,Monday,3,1,2011,d_3,none,none,none,...,31,0,31,True,False,False,False,False,False,1296432000
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,none,none,none,...,1,1,32,False,True,False,False,False,False,1296518400
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,none,none,none,...,2,2,33,False,False,False,False,False,False,1296604800


### Feature Engineering - sales

In [18]:
#merge with sales data
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

In [5]:
sales_df = pd.read_pickle(data_path/'sales_df')
calendar_df = pd.read_pickle(data_path/'calendar_df') 

In [131]:
sales_df.to_pickle(data_path/'sales_df')

In [132]:
calendar_df.to_pickle(data_path/'calendar_df')

In [ ]:
sales_df = join_df(sales_df, calendar_df, 'd', 'd')
sales_df.head()

In [ ]:
data_path/'calendar_temp.csv'

### Merging Sales and Calendar

I keep running out of memory trying to merge the dataset, so I wrote the dataframes to disk. Then merged them using chunking.

In [17]:
#read pickle file
sales_df = pd.read_pickle(data_path/'sales_df')
calendar_df = pd.read_pickle(data_path/'calendar_df')

#convert to csv
sales_df.to_csv(data_path/'sales_temp.csv', index_label=False)
calendar_df.to_csv(data_path/'calendar_temp.csv', index_label=False)

#create a temp file with just the header
df_result = pd.DataFrame(columns=(sales_df.columns.append(calendar_df.columns)).unique())
df_result.to_csv("merge.csv", index_label=False)

#delete sales so save memory
del(sales_df)

#func to process each chunk
def preprocess_merge(x):
    df2 = join_df(x, calendar_df, 'd', 'd')
    df2.to_csv(data_path/'merge.csv', mode="a", header=False, index=False)

#chunk merge.csv and run preprocess func on each chunk
reader = pd.read_csv(data_path/'sales_temp.csv', chunksize=100000)
[preprocess_merge(r) for r in reader]

In [23]:
#make sure everything looks good
merged_reader = pd.read_csv(data_path/'merge.csv', chunksize=100000)
temp = next(merged_reader).copy(deep=True)
temp.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,value,date,wm_yr_wk,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,29,5,29,False,False,False,False,False,False,1296259200
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,29,5,29,False,False,False,False,False,False,1296259200
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,29,5,29,False,False,False,False,False,False,1296259200
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,29,5,29,False,False,False,False,False,False,1296259200
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,29,5,29,False,False,False,False,False,False,1296259200


In [ ]:
#setup elasped

In [25]:
# cumulative counting across a sorted df
def get_elapsed(fld, pre, df, reset_on='store_id'):
    day1 = np.timedelta64(1, 'D')
    last_date = np.datetime64()
    last_store = 0
    res = []

    for s,v,d in zip(df[reset_on].values,df[fld].values, df.date.values):
        if s != last_store:
            last_date = np.datetime64()
            last_store = s
        if v: last_date = d
        res.append(((d-last_date).astype('timedelta64[D]') / day1))
    df[pre+fld] = res

In [26]:
columns = ["date", "store_id", "event_name_1", "event_name_2"]

### Repeat for Item Prices

Repeat the process to merge the data with item prices.

In [30]:
#read pickle file
sales_df = pd.read_pickle(data_path/'sales_df')
calendar_df = pd.read_pickle(data_path/'calendar_df')
prices_df = pd.read_csv(data_path/'sell_prices.csv')

#create a temp file with just the header
df_result = pd.DataFrame(columns=(sales_df.columns.append(calendar_df.columns).append(prices_df.columns)).unique())
df_result.to_csv("merge_2.csv", index_label=False)

In [31]:
del(sales_df)

In [4]:
prices_df = pd.read_csv(data_path/'sell_prices.csv')

In [6]:
#join based on id
# HOBBIES_1_001_CA_1_validation -> {item_id}_{store_id}_validation
prices_df['merge_col'] = prices_df.apply(lambda x: f'{x["item_id"]}_{x["store_id"]}_validation', axis=1)

In [8]:
prices_df.to_csv(data_path/'prices_temp.csv', index_label=False)

In [10]:
prices_temp = pd.read_csv(data_path/'prices_temp.csv')

In [27]:
y[y['sell_price'].isnull() == False]

,id,item_id,dept_id,cat_id,store_id,state_id,d,value,date,wm_yr_wk,...,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed,merge_col,sell_price
7,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12,2011-01-29,11101,...,29,False,False,False,False,False,False,1296259200,HOBBIES_1_008_CA_1_validation,0.46
8,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2,2011-01-29,11101,...,29,False,False,False,False,False,False,1296259200,HOBBIES_1_009_CA_1_validation,1.56
9,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,29,False,False,False,False,False,False,1296259200,HOBBIES_1_010_CA_1_validation,3.17
11,HOBBIES_1_012_CA_1_validation,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,29,False,False,False,False,False,False,1296259200,HOBBIES_1_012_CA_1_validation,5.98
14,HOBBIES_1_015_CA_1_validation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,d_1,4,2011-01-29,11101,...,29,False,False,False,False,False,False,1296259200,HOBBIES_1_015_CA_1_validation,0.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,HOUSEHOLD_1_436_CA_1_validation,HOUSEHOLD_1_436,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,d_1,2,2011-01-29,11101,...,29,False,False,False,False,False,False,1296259200,HOUSEHOLD_1_436_CA_1_validation,10.97
992,HOUSEHOLD_1_437_CA_1_validation,HOUSEHOLD_1_437,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,d_1,6,2011-01-29,11101,...,29,False,False,False,False,False,False,1296259200,HOUSEHOLD_1_437_CA_1_validation,3.28
993,HOUSEHOLD_1_438_CA_1_validation,HOUSEHOLD_1_438,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,d_1,4,2011-01-29,11101,...,29,False,False,False,False,False,False,1296259200,HOUSEHOLD_1_438_CA_1_validation,6.56
994,HOUSEHOLD_1_439_CA_1_validation,HOUSEHOLD_1_439,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,d_1,2,2011-01-29,11101,...,29,False,False,False,False,False,False,1296259200,HOUSEHOLD_1_439_CA_1_validation,0.94


In [36]:
#adding sell_price by merging merge.csv and sales.csv

for chunk in pd.read_csv(data_path/'merge.csv', chunksize=100000):
    df = join_df(chunk, prices_temp[['merge_col', 'wm_yr_wk', 'sell_price']], 
            ['id', 'wm_yr_wk'], ['merge_col', 'wm_yr_wk'])
    df.to_csv(data_path/'merge_2.csv', mode="a", header=False, index=False)

# fld = 'event_name_1'
# df = test.sort_values(['store_id', 'date'])
# get_elapsed(fld, 'After', df)
# df = df.sort_values(['store_id', 'date'], ascending=[True, False])
# get_elapsed(fld, 'Before', df)
# df

In [113]:
test = calendar_df.copy(deep=True)

In [114]:
fld = 'event_name_1'
df = test.sort_values(['store_id', 'date'])
get_elapsed(fld, 'After', df)
df = df.sort_values(['store_id', 'date'], ascending=[True, False])
get_elapsed(fld, 'Before', df)
df

KeyError: 'store_id'